In [1]:
import os
import json
from collections import deque
from typing import Dict, List, Optional, Any, Union

from langchain.chat_models import ChatOpenAI


# Get human objective
# refine it via human in the loop and via adding context (short/long term memory search) -> the output is a json dict of the design
# build the modules and test benches usiiing short and long term memory

In [2]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")

In [3]:
llm = ChatOpenAI(model='gpt-4', temperature=0)
bigllm = ChatOpenAI(model='gpt-4-1106-preview', temperature=0)

In [4]:
import FPGA_AGI.agents

## FPGA-AGI

In [ ]:
fpga_agi_instance = FPGA_AGI.agents.FPGA_AGI.from_llm(llm=bigllm, verbose=True)

objective = """
{
    "Module_Name": "state_machine",
    "ports": ["input clk", "input enable", "output 4 bit count"],
    "description": "A counter that counts from 1 to 12 and has an enable signal",
    "connections": ["None"],
    "hdl_language": "Verilog"
}
"""
for i in range(10):
    fpga_agi_instance(objective=objective ,action_type='hdl')

In [ ]:
fpga_agi_instance = FPGA_AGI.agents.FPGA_AGI.from_llm(llm=bigllm, verbose=True)

objective = """
{
    "Module_Name": "8_adder",
    "ports": ["input 8 bit a", "input 8 bit b", "output 8 bit result", "output 1 bit over_flow"],
    "description": "signed 8 bit adder with over flow and no clock",
    "connections": ["None"],
    "hdl_language": "Verilog"
}
"""
for i in range(10):
    fpga_agi_instance(objective=objective ,action_type='hdl')

In [ ]:
fpga_agi_instance = FPGA_AGI.agents.FPGA_AGI.from_llm(llm=bigllm, verbose=True)

objective = """
{
    "Module_Name": "shiftreg_64",
    "ports": ["input clk", "input 64 bit data_in", "input load", "input 1 bit shift_direction", "output 64 bit data_out", "output 1 bit over_flow"],
    "description": "64 bit arithmatic shift register with overflow indicator and data input/output. This arithmatic shift register is a signed shift register. the direction of the shift is controlled via shift_direction such that 0 is left and 1 is right shift. data_in is the input of the shift register before the shift. when the load signal in on the value of the shift register is set to data_in and otherwise the shift will be taking place.",
    "connections": ["None"],
    "hdl_language": "Verilog"
}
"""
for i in range(10):
    fpga_agi_instance(objective=objective ,action_type='hdl')

In [ ]:
fpga_agi_instance = FPGA_AGI.agents.FPGA_AGI.from_llm(llm=bigllm, verbose=True)

objective = """
{
    "Module_Name": "state_machine",
    "ports": ["input clk", "input rst", "input stream", "output 1 bit detect"],
    "description": "A sequence detector that detects 101 when it arrives at the input stream. A sequence detector's functions are achieved by using a finite state machine.",
    "connections": ["None"],
    "hdl_language": "Verilog"
}
"""
for i in range(10):
    fpga_agi_instance(objective=objective ,action_type='hdl')
#fpga_agi_instance(objective="design a very simple 32 bit risc-v cpu in verilog. The cpu will be a softcore solution for zync ultrascale devices and it must connect to the PS side of the FPGA via AXI-stream for the data and AXI-lite for the control. As a note, I am going to use pynq to connect to this device so these axi interfaces must be pynq compatible as much as possible." ,action_type='module')

In [ ]:
fpga_agi_instance = FPGA_AGI.agents.FPGA_AGI.from_llm(llm=bigllm, verbose=True)

objective = """
{
    "Module_Name": "ABRO_FSM",
    "ports": ["input clk", "input rst", "input A", "input B", "output R"],
    "description": "an FSM where the states represent the waiting for signals A and B, and the transitions represent the reception of these signals. The FSM would ultimately reach a state where it can emit the R signal, indicating that both A and B have been received. the state machine stays in state R until a rst signal is received. Make sure you write in verilog and do not use system verilog syntax.",
    "connections": ["None"],
    "hdl_language": "Verilog"
}
"""
for i in range(10):
    fpga_agi_instance(objective=objective ,action_type='hdl')

In [ ]:
fpga_agi_instance = FPGA_AGI.agents.FPGA_AGI.from_llm(llm=bigllm, verbose=True)
objective = """
I want you to design and code a working 32 bit RISC-V CPU with the following instruction set:
| Instruction | Format | Meaning                       |
|-------------|--------|-------------------------------|
| add         | R      | Add registers                 |
| sub         | R      | Subtract registers            |
| sll         | R      | Shift left logical by register|
| srl         | R      | Shift right logical by register|
| sra         | R      | Shift right arithmetic by register|
| and         | R      | Bitwise AND with register     |
| or          | R      | Bitwise OR with register      |
| xor         | R      | Bitwise XOR with register     |
| slt         | R      | Set if less than register, 2's complement|
| sltu        | R      | Set if less than register, unsigned|
| addi        | I      | Add immediate                 |
| slli        | I      | Shift left logical by immediate|
| srli        | I      | Shift right logical by immediate|
| srai        | I      | Shift right arithmetic by immediate|
| andi        | I      | Bitwise AND with immediate    |
| ori         | I      | Bitwise OR with immediate     |
| xori        | I      | Bitwise XOR with immediate    |
| slti        | I      | Set if less than immediate, 2's complement|
| sltiu       | I      | Set if less than immediate, unsigned|
| lui         | U      | Load upper immediate          |
| auipc       | U      | Add upper immediate to pc     |

Include the above table in the requirement.
The CPU will be implemented on a ultra-96 v2 board.
We will imlement the communication protocols separately so do not worry about them.
The CPU must have separate instruction and data memory.
"""
fpga_agi_instance(objective=objective ,action_type='full')
#fpga_agi_instance.requirement_agent_executor.run(objective)

In [ ]:
fpga_agi_instance = FPGA_AGI.agents.FPGA_AGI.from_llm(llm=bigllm, verbose=True)
objective = """
design an 4 point FFT hardware in verilog such that you have 4 32 bit inputs and 4 32 bit outputs. each 32 bit consists of 16 bits (MSB) real and 16 bits (LSB) imaginary part.
The solution must use two 2_point FFTs to build the 8 point FFT and it should consist of the following modules: two 2_point FFT modules, butterfly module and complex multiplication module.
use the available tools to learn to compute the twiddle factors accurately (this is a very important piece of information). do not make them up on your own. 
"""
for i in range(10):
    fpga_agi_instance(objective=objective ,action_type='full')

In [5]:
fpga_agi_instance = FPGA_AGI.agents.FPGA_AGI.from_llm(llm=llm, verbose=True)
objective = """
Write a four-point DIF-FFT verilog code. You can use the following IP cores to build the target four-point FT IP core.

Here is the template of butterfly computation IP Core,
butterfly_computation butterfly_inst(.clk(),.reset(),.enable(),.in1(), .in2(),.done(),.out1(), .out2());
And here is the template of the two-point FFT IP Core,
fft_2_point fft_2_point_inst(.clk(),.reset(),.enable(),.x1(), .x2(),.done(),.y1(), .y2());
And here is the template of complex multiplication IP Core,
complex_multiplication complex_multiplication_inst(.clk(),.reset(),.enable(),.in1(), .in2(),.done(),.out());

Here's the template of the target four-point FFT IP Core,
fft_4_point fft_4_point_stage_2_1(.clk(),.reset(),.enable(),.x1(), .x2(), .x3(), .x4(),.y1(), .y2(), .y3(), .y4(),.done());

Do not forget clock, reset and done!
Do not build the fft_2_point module. Just assume that it exists and instantiate it wherever needed.

The input and output signals (x's and y's) have 16 bits in the real part (MSB) and
6 16 bits in the imaginary part (LSB). Further, I also have some suggestions for you.

First, you need to calculate the twiddle factors for a four-point DIF-FFT using the tools you are given. Do not forget the scalings and remember that the range for 16 bit signed integers is [-32767, 32768]

Second, a 4-point FFT consists of two stages.
In the first stage of FFT, you have two substages called Stage 1a and 1b.
In stage 1a, you use two butterfly_computation modules; and then in stage 1b,
you use two twiddle models. In the second stage, you are facing a two
two-point FFT problem, so you just use two fft_2_point and solve the problem.

"""
for i in range(5):
    try:
        fpga_agi_instance(objective=objective ,action_type='full')
    except:
        print("We fucked up this round. Moving to the next!")



> Entering new RequirementAgentExecutor chain...
Thought: The question is asking for a comprehensive plan for the development of a four-point DIF-FFT verilog code using the provided IP cores. The first step is to extract the main goals based on the objective.

Goals: 
1. Develop a four-point DIF-FFT verilog code using the provided IP cores.
2. Calculate the twiddle factors for a four-point DIF-FFT.
3. Implement the two stages of the 4-point FFT.

Thought: Now that the goals have been identified, the next step is to detail the technical requirements and constraints of the project.

Requirements: 
1. Use the provided IP cores to build the target four-point FT IP core.
2. Calculate the twiddle factors for a four-point DIF-FFT using the tools provided. Remember the scalings and the range for 16 bit signed integers is [-32767, 32768].
3. Implement the two stages of the 4-point FFT. In the first stage, use two butterfly_computation modules in stage 1a and two twiddle models in stage 1b. In

In [5]:
import re
re.compile(r"[\*\+\-\s]*\bgoals\b\s*:*", re.IGNORECASE)

def extract_project_details(text):
    lines = text.split('\n')
    goals, requirements, constraints = "", "", ""
    current_section = None
    sections = {
        re.compile(r"[\*\+\-\s]*\bGoals\b\s*:*", re.IGNORECASE): "goals",
        re.compile(r"[\*\+\-\s]*\bRequirements\b\s*:*", re.IGNORECASE): "requirements",
        re.compile(r"[\*\+\-\s]*\bConstraints\b\s*:*", re.IGNORECASE): "constraints"
    }
    for line in lines:
        for section_regex in sections:
            if section_regex.match(line):
                current_section = sections[section_regex]
                line = line.split(":", 1)[1] if ":" in line else line  # Remove the header part
                break

        if current_section:
            if current_section == "goals":
                goals += line + "\n"
            elif current_section == "requirements":
                requirements += line + "\n"
            elif current_section == "constraints":
                constraints += line + "\n"
    return goals, requirements, constraints

text = """
- Goals: 
1. Develop a four-point DIF-FFT verilog code using the provided IP cores.
2. Calculate the twiddle factors for a four-point DIF-FFT.
3. Implement the two stages of the 4-point FFT.

Thought: Now that the goals have been identified, the next step is to detail the technical requirements and constraints of the project.

- Requirements: 
1. Use the provided IP cores to build the target four-point FT IP core.
2. Calculate the twiddle factors for a four-point DIF-FFT using the tools provided. Remember the scalings and the range for 16 bit signed integers is [-32767, 32768].
3. Implement the two stages of the 4-point FFT. In the first stage, use two butterfly_computation modules in stage 1a and two twiddle models in stage 1b. In the second stage, use two fft_2_point modules.
4. The input and output signals (x's and y's) should have 16 bits in the real part (MSB) and 16 bits in the imaginary part (LSB).

- **Constraints: 
The ASCII Diagram of the potential design is as follows:
"""

goals, requirements, constraints = extract_project_details(text)

constraints


' \nThe ASCII Diagram of the potential design is as follows:\n\n'